In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("data/raw/pitchdata.csv", sep=',')

In [11]:
# For Pitcher team Id
data_pitch = df.groupby(['PitcherTeamId','HitterSide'],as_index = False)['PA','AB','H','TB','BB','SF','HBP'].sum()
data_pitch.columns = ['SubjectId' if x=='PitcherTeamId' else x for x in data_pitch.columns]
data_pitch['Split'] = data_pitch.HitterSide.apply(lambda x: "vs LHH" if x == 'L' else 'vs RHH')
data_pitch['Subject'] = 'PitcherTeamId'

In [12]:
# For Pitcher Id
data_pitch_id = df.groupby(['PitcherId','HitterSide'],as_index = False)['PA','AB','H','TB','BB','SF','HBP'].sum()
data_pitch_id.columns = ['SubjectId' if x=='PitcherId' else x for x in data_pitch_id.columns]
data_pitch_id['Split'] = data_pitch_id.HitterSide.apply(lambda x: "vs LHH" if x == 'L' else 'vs RHH')
data_pitch_id['Subject'] = 'PitcherId'

In [13]:
# For Hitter team Id
data_hitter = df.groupby(['HitterTeamId','PitcherSide'],as_index = False)['PA','AB','H','TB','BB','SF','HBP'].sum()
data_hitter.columns = ['SubjectId' if x=='HitterTeamId' else x for x in data_hitter.columns]
data_hitter['Split'] = data_hitter.PitcherSide.apply(lambda x: "vs LHP" if x == 'L' else 'vs RHP')
data_hitter['Subject'] = 'HitterTeamId'

In [14]:
# For Hitter team
data_hitter_id = df.groupby(['HitterId','PitcherSide'],as_index = False)['PA','AB','H','TB','BB','SF','HBP'].sum()
data_hitter_id.columns = ['SubjectId' if x=='HitterId' else x for x in data_hitter_id.columns]
data_hitter_id['Split'] = data_hitter_id.PitcherSide.apply(lambda x: "vs LHP" if x == 'L' else 'vs RHP')
data_hitter_id['Subject'] = 'HitterId'

In [20]:
# Combining Hitter and Picther team
features = ['SubjectId','Split','Subject','PA','AB','H','TB','BB','SF','HBP']
data_combined = pd.concat([data_pitch[features],data_hitter[features],data_pitch_id[features],data_hitter_id[features]],axis = 0)

In [22]:
data_combined.shape

(2310, 10)

In [23]:
data_final = data_combined[data_combined['PA']>=25]

In [27]:
data_final.head()

,SubjectId,Split,Subject,PA,AB,H,TB,BB,SF,HBP,AVG,OBP,SLG,OPS
0,108,vs LHH,PitcherTeamId,175,161,36,58,12,1,1,0.224,0.280,0.360,0.640
1,108,vs RHH,PitcherTeamId,445,393,107,155,42,5,5,0.272,0.346,0.394,0.740
2,109,vs LHH,PitcherTeamId,298,262,60,95,33,1,1,0.229,0.316,0.363,0.679
3,109,vs RHH,PitcherTeamId,325,290,65,115,25,4,4,0.224,0.291,0.397,0.688
4,110,vs LHH,PitcherTeamId,228,202,45,75,19,1,3,0.223,0.298,0.371,0.669


In [26]:
# Calculating Stats
data_final['AVG'] = (data_final.H/ data_final.AB).round(3)
data_final['OBP'] = ((data_final.H + data_final.BB + data_final.HBP) / (data_final.AB + data_final.BB + data_final.HBP + data_final.SF)).round(3)
data_final['SLG'] = (data_final.TB / data_final.AB).round(3)
data_final['OPS'] = data_final.SLG + data_final.OBP

D:\Anaconda\envs\py36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
D:\Anaconda\envs\py36\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
D:\Anaconda\envs\py36\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

In [28]:
# Getting the desired shape
data_final = pd.melt(data_final, id_vars = ['SubjectId','Split','Subject'], value_vars = ['AVG','OBP','SLG','OPS'], var_name = 'Stat',value_name = 'Value')

In [31]:
# Exporting
data_final.sort_values(by = ['SubjectId','Stat','Split','Subject'])[['SubjectId','Stat','Split','Subject','Value']].to_csv('./data/processed/output.csv',index = False)